# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Драпак С. Н.

Группа: 317

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [5]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(data.iloc[:, 1:]), np.array(data.iloc[:, 0]),
                                                    test_size=0.3, random_state=241)

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния (учитите, что он должен возвращать вероятность — отношение объектов первого класса среди соседей к числу соседей). Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba).

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

In [7]:
#test = np.array([[1, 1, 3], [1, 3, 3], [3, 3, 2], [1, 1, 3]])


def f_1d(x, i):
    return int(dicts[i].setdefault(x, 0))


# Строит матрицу f. Вообще, функцкия немного избыточна,
# Но вводить словари я решил не сразу и поскольку она вычисляется быстро,
# то строил словарь на ее основе
def f(X):
    X_t = np.transpose(X)
    res = np.ndarray((0, X.shape[0]))
    for x in X_t:
        tmp_res = np.ndarray(x.shape[0])
        for i in xrange(x.shape[0]):
            tmp_res[i] = len(np.where(x == x[i])[0])
        res = np.vstack((res, tmp_res))
   
    return np.transpose(res)

# Вычисляет словарь f
def creat_dict_of_f(X):
    global dicts
    dicts = np.array([dict() for x in range(X.shape[1])])
    f_X = f(X)
    for i in xrange(dicts.shape[0]):
        for j in xrange(X.shape[0]):
            dicts[i].setdefault(X[j, i], f_X[j, i])
    #print dicts
    return dicts


# Словарь p^2
def create_dict_of_p2(X):
    global p2_dicts
    p2_dicts = np.array([dict() for x in range(X.shape[1])])
    for i in range(X.shape[1]):
        for j in range(p2_matrix.shape[0]):
            p2_dicts[i].setdefault(X[j, i], p2_matrix[j, i])
    return p2_dicts

            
#f_matrix = f(test)

# Строит матрицу P
def p_compute(X):
    res = np.ndarray((0, X.shape[0]))
    X_t = np.transpose(X)
    k = 0
    for x in X_t:
        res = np.vstack((res, f_matrix[:, k]/x.shape[0]))
        k += 1
    return np.transpose(res)


# Строит матрицу p
def p2_compute(X):
    X_t = np.transpose(X)
    res = p_compute(X)/(X.shape[0] - 1)
    tmp = np.ndarray((0, X.shape[0]))
    k = 0
    for x in X_t:
        tmp = np.vstack((tmp, (f_matrix[:, k] - 1)))
        k += 1
    tmp = np.transpose(tmp)
    return res * tmp
    
     
#p_matrix = p_compute(test)
#p2_matrix = p2_compute(test)
#print p2_matrix
     
    

In [8]:
# Функция считает попарные расстояния вида Di(x, y) = [xi = yj]*log(fi(xi)) * log(fi(yi))

def d3_cnb(X, Y, n_nb):
    Y_f_matrix = np.ndarray(Y.shape)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y_f_matrix[i, j] = f_1d(Y[i, j], j)
    distance_matrix = ((X[:, np.newaxis, :] != Y[np.newaxis, :, :]) * np.log(f_matrix[:, np.newaxis, :] + 1) * np.log(Y_f_matrix[np.newaxis, :, :] + 1)).sum(axis=2)
    distance_matrix = (np.transpose(distance_matrix))
    return np.argpartition(distance_matrix, n_nb - 1, axis = 1)[:, 0:n_nb]
    

In [9]:
# Функция создает словарь из сумм по матрице p^2 для всех элементов матрицы X
# (сумм из формулы для сглаженного индикатора совпадений)

def create_p2_sum(X):
    global p2_sum
    p2_sum = np.array([dict() for x in range(X.shape[1])])
    for i in xrange(X.shape[1]):
        for key_2 in dicts[i].keys():
            t_sum = 0
            for key in dicts[i].keys():
                if dicts[i][key] <= dicts[i][key_2]:
                    t_sum += p2_dicts[i][key]
            p2_sum[i][key_2] = t_sum
    return p2_sum

In [10]:
# Считает матрицу попарных расстояний сглаженого индикатора совпадений
def smooth_cnb(X, Y, n_nb=10):
    #print "HERE"
    
    #print X.shape
    #print Y.shape
    distance_matrix = ((X[:, np.newaxis, :] != Y[np.newaxis, :, :]).sum(axis=2)).astype(float)
    
    p2_X = np.ndarray(X.shape)
    
              
    
    for i in xrange(X.shape[0]):
        for j in xrange(X.shape[1]):
                p2_X[i, j] = p2_sum[j][X[i, j]]
    
    #print p2_X.shape
    #print ((X[:, np.newaxis, :] == Y[np.newaxis, :, :]) * p2_X[:, np.newaxis, :]).sum(axis=2)
    distance_matrix += ((X[:, np.newaxis, :] == Y[np.newaxis, :, :]) * p2_X[:, np.newaxis, :]).sum(axis=2)
           
    distance_matrix = np.transpose(distance_matrix)
    return np.argpartition(distance_matrix, n_nb - 1, axis = 1)[:, 0:n_nb]


In [11]:
# Вычисляет матрицу попарных расстояний для индикаторного расстояния

def ind_cnb(X, Y, n_nb):
    distance_matrix = (X[:, np.newaxis, :] != Y[np.newaxis, :, :]).sum(axis=2)
    distance_matrix = np.transpose(distance_matrix)
    return np.argpartition(distance_matrix, n_nb - 1, axis = 1)[:, 0:n_nb]

In [11]:
# Функция взята из задания по практикуму, поэтому немного избыточна.
# Она делит тестовую выборку на несколько частей и ищет соседей

def divide_and_find_nb(X_train, X_test, n_nb, func_fit, func_find, part=10, return_distance=False):
    cn = np.ndarray((0, n_nb), dtype=int)
    distance_matrix = np.ndarray((0, n_nb), dtype=float)
    if (func_fit != func_find):
        func_fit(X_train)
        
    for p in xrange(part):
        if (func_fit == func_find):
            cn = np.vstack((cn, func_find(X_train,
                                          X_test[p * X_test.shape[0]/part : (p + 1) * X_test.shape[0]/part],
                                          n_nb)))
        else:
            if (return_distance):
                sub_distance_matrix, _ = func_find(X_test[p * X_test.shape[0]/part : (p + 1) * X_test.shape[0]/part],
                                          n_nb, return_distance=True)
                distance_matrix = np.vstack((distance_matrix, sub_distance_matrix))
            
            cn = np.vstack((cn, func_find(X = X_test[p * X_test.shape[0]/part : (p + 1) * X_test.shape[0]/part],
                                          n_neighbors = n_nb, return_distance=False)))
            
            
    if (return_distance):
        return distance_matrix, cn
    return cn

In [19]:
import scipy.spatial
import sys
   
# Реализация knn
def my_knn(X_train, y_train, X_test, metric='ind', n_nb=10):
    res = np.ndarray(X_test.shape[0], dtype = float)
    nb_matrix = 0
    
    if (metric == 'ind'):
        nb_matrix = divide_and_find_nb(X_train, X_test, n_nb, ind_cnb, ind_cnb)
    
    if (metric != 'ind'):
        creat_dict_of_f(X_train)
        
    
    if (metric == 'smooth_ind'):
        global global_train
        global_train = X_train
        global f_matrix 
        f_matrix = f(X_train)
        global p_matrix
        p_matrix = p_compute(X_train)
        global p2_matrix
        p2_matrix = p2_compute(X_train)
        create_dict_of_p2(X_train)
        create_p2_sum(X_train)
        
        #metric_f = smooth_indicator_distance
        nb_matrix = divide_and_find_nb(X_train, X_test, n_nb, smooth_cnb, smooth_cnb)
    
        
    if (metric == 'd3'):
        global f_matrix 
        f_matrix = f(X_train)
        global global_train
        global_train = X_train
        nb_matrix = divide_and_find_nb(X_train, X_test, n_nb, d3_cnb, d3_cnb)
        
    #print nb_matrix.shape
    
    print "Preparing computed"
    sys.stdout.flush()
    
    #nb_matrix = closest_nb(X_train, X_test, n_nb, metric_f)
    
    print "Distance computed"
    
    #print nb_matrix.shape
    
    for i in xrange(nb_matrix.shape[0]):
        classes = y_train[nb_matrix[i]]
        cnt = np.bincount(classes)
        if len(cnt) == 1:
            res[i] = 0
            continue        
        res[i] = float(cnt[1])/np.sum(cnt)
    return res      
    



<ipython-input-19-4ec1d52a3640>:33: SyntaxWarning: name 'f_matrix' is assigned to before global declaration
  global f_matrix
<ipython-input-19-4ec1d52a3640>:35: SyntaxWarning: name 'global_train' is assigned to before global declaration
  global global_train


Вычисление точности с индикаторным расстоянием

In [45]:
res = my_knn(X_train, y_train, X_test, metric='ind', n_nb=10)
#print accuracy(res, y_test)

Preparing computed
Distance computed


In [67]:
import sklearn.metrics
print sklearn.metrics.roc_auc_score(y_test, res)

0.829499015542


Вычисление точности со сглаженным индикаторным расстоянием

In [16]:
res_smooth = my_knn(X_train, y_train, X_test, metric='smooth_ind', n_nb=10)

Preparing computed
Distance computed


In [17]:
import sklearn.metrics
print sklearn.metrics.roc_auc_score(y_test, res_smooth)

0.833094317461


Вычисление качества логорифмического расстояния

In [20]:
import sklearn.metrics
res_3 = my_knn(X_train, y_train, X_test, metric='d3', n_nb=10)
print sklearn.metrics.roc_auc_score(y_test, res_3)

Preparing computed
Distance computed
0.821046030503


В итоге сглаженное расстояние получилось самым сильным. Сложно сказать, почему именно так, нужно более внимательно изучать признаки.

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

 Считаем счетчики клики и их отношение.
 Тут хочется прокоментировать один момент. В задании было написано, что для тестовой выборки
 нужно считать клики по всему трейну, но ведь в такос случие, без доп. обработки мы получим,
 что значение кликов и каунтов на тесте будет в среднем значительно выше при n = 3.
 Поскольку никакой речи о нормировки и прочих вещах в задании не было, я выбирал случайные N - N/n объектов из трейна
 (именно по такому числу мы считаем счетчики для каждого объекта из трейна) и считал их для теста. 

In [38]:
from numpy.random import shuffle


def categorial_feat_transorm(x, y, x_test, n_parts = 1):
    #print "HERE"
    counts = np.ndarray(x.shape[0])
    clicks = np.ndarray(x.shape[0])
    smooth = np.ndarray(x.shape[0])
    
    counts_test = np.ndarray(x_test.shape[0])
    clicks_test = np.ndarray(x_test.shape[0])
    smooth_test = np.ndarray(x_test.shape[0])
    
    k = x.shape[0]/n_parts
    for part in range(n_parts):
        x_part = x[part * k : (part + 1) * k]
        y_part = y[part * k : (part + 1) * k]
        
        if (n_parts == 1):
            rest_ind = range(0, x.shape[0])
        else:
            rest_ind = range(0, part * k) + range((part + 1) * k , y.shape[0])
        #print rest_ind
        #print x.shape
        x_rest = x[rest_ind]
        y_rest = y[rest_ind]
        #print x_rest
        #print x_part
        
        for i in range(part * k, (part + 1) * k):
            #print x[i].shape
            #print x_rest, x_rest == x[i]
            #print i, x[i]
            counts[i] = len(np.where(x == x[i])[0])
            clicks[i] = len(np.where((x_rest == x[i]) & (y_rest == 1))[0])
            
    smooth = (clicks + 1).astype(float)/(counts + 2).astype(float)
    random_indces = range(0, x.shape[0])
    if (n_parts != 1):
        np.random.shuffle(random_indces)
        random_indces = random_indces[:x.shape[0] - k]
    #print random_indces
    for i in range(x_test.shape[0]):
        counts_test[i] = len(np.where(x == x_test[i])[0])
        clicks_test[i] = len(np.where((x[random_indces] == x_test[i]) & (y[random_indces] == 1))[0])
        smooth_test[i] = float(clicks_test[i] + 1)/float(counts_test[i] + 2)
    
    return np.transpose(np.vstack((counts, clicks, smooth))), np.transpose(np.vstack((counts_test, clicks_test, smooth_test)))   
    

Считаем клики без фолдов

In [41]:
New_Train = np.ndarray((X_train.shape[0], 0), dtype=float)
New_Test = np.ndarray((X_test.shape[0], 0), dtype=float)
for i in xrange(X_train.shape[1]):
    #print New_Train.shape, categorial_feat_transorm(x, y_train, 3).shape
    t_train, t_test = categorial_feat_transorm(X_train[:, i], y_train, X_test[:, i], 1)
    New_Train = np.hstack((New_Train, t_train))
    New_Test = np.hstack((New_Test, t_test))
    
    
from sklearn.preprocessing import normalize
#New_Test = normalize(New_Test)
#New_Train = normalize(New_Train)

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
for k in range(1, 20):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(New_Train, y_train)
    t_res = neigh.predict_proba(New_Test)[:, 1]
    #print t_res.shape, y_test.shape
    print "k = " + str(k)
    print roc_auc_score(y_test, t_res)

k = 1
0.712921090257
k = 2
0.752472646161
k = 3
0.775740481957
k = 4
0.777982378959
k = 5
0.776867540638
k = 6
0.784620101343
k = 7
0.787899830425
k = 8
0.789534842763
k = 9
0.78571687869
k = 10
0.78870008432
k = 11
0.788626223024
k = 12
0.791252971524
k = 13
0.790561532749
k = 14
0.796091066126
k = 15
0.794659936123
k = 16
0.794899311419
k = 17
0.793385514272
k = 18
0.791875850483
k = 19
0.790121599775


С фолдами

In [187]:
New_Train = np.ndarray((X_train.shape[0], 0), dtype=float)
New_Test = np.ndarray((X_test.shape[0], 0), dtype=float)
for i in xrange(X_train.shape[1]):
    #print New_Train.shape, categorial_feat_transorm(x, y_train, 3).shape
    t_train, t_test = categorial_feat_transorm(X_train[:, i], y_train, X_test[:, i], 3)
    New_Train = np.hstack((New_Train, t_train))
    New_Test = np.hstack((New_Test, t_test))
    
    
from sklearn.preprocessing import normalize
#New_Test = normalize(New_Test)
#New_Train = normalize(New_Train)

In [188]:
for k in range(1, 20):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(New_Train, y_train)
    t_res = neigh.predict_proba(New_Test)[:, 1]
    #print t_res.shape, y_test.shape
    print "k = " + str(k)
    print roc_auc_score(y_test, t_res)

k = 1
0.667906943393
k = 2
0.702428742719
k = 3
0.733860498116
k = 4
0.754097414962
k = 5
0.760161858674
k = 6
0.76526583597
k = 7
0.768760859048
k = 8
0.771730945762
k = 9
0.773286525759
k = 10
0.77024742995
k = 11
0.771800314279
k = 12
0.772076350655
k = 13
0.774218507952
k = 14
0.7761011623
k = 15
0.775032330042
k = 16
0.77916487854
k = 17
0.780158232102
k = 18
0.782616411513
k = 19
0.782001260135


Получаем, что КНН все же лучше пока обучается без фолдов

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

Функция создает новые параметры по парам. Я решил, что проще и эффективней всего считать хэши от 2х признаков их конкатенацией.

In [16]:
def categorial_feat_concat(X):
    new_feat = X
    for i in xrange(X.shape[1]):
        for j in xrange(i):
            tmp = np.ndarray((X.shape[0], 1), dtype=int)
            #print (type(tmp[1]))
            for k in xrange(X.shape[0]):
                #print k, i, j
                #print hash((X[k, i], X[k, j]))
                tmp[k][0] = hash((X[k, i], X[k, j]))
            #print new_feat.shape, tmp.shape
            
            new_feat = np.hstack((new_feat, tmp))
    return new_feat
    

С конкатенацией, но без фолдов

In [43]:
New_train_hash = categorial_feat_concat(X_train)
New_test_hash = categorial_feat_concat(X_test)
print New_train_hash.shape

counts_hash_train = np.ndarray((New_train_hash.shape[0], 0), dtype=float)
counts_hash_test = np.ndarray((New_test_hash.shape[0], 0), dtype=float)
for i in xrange(X_train.shape[1]):
    #print New_Train.shape, categorial_feat_transorm(x, y_train, 3).shape
    t_train, t_test = categorial_feat_transorm(New_train_hash[:, i], y_train, New_test_hash[:, i], 1)
    counts_hash_train = np.hstack((counts_hash_train, t_train))
    counts_hash_test = np.hstack((counts_hash_test, t_test))



(22938L, 45L)


In [44]:
#print counts_hash_test.shape
for k in range(1, 20):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(counts_hash_train, y_train)
    t_res = neigh.predict_proba(counts_hash_test)[:, 1]
    #print t_res.shape, y_test.shape
    print "k = " + str(k)
    print roc_auc_score(y_test, t_res)

k = 1
0.712921090257
k = 2
0.752472646161
k = 3
0.775740481957
k = 4
0.777982378959
k = 5
0.776867540638
k = 6
0.784620101343
k = 7
0.787899830425
k = 8
0.789534842763
k = 9
0.78571687869
k = 10
0.78870008432
k = 11
0.788626223024
k = 12
0.791252971524
k = 13
0.790561532749
k = 14
0.796091066126
k = 15
0.794659936123
k = 16
0.794899311419
k = 17
0.793385514272
k = 18
0.791875850483
k = 19
0.790121599775


С фолдами, и с конкатенацией

In [ ]:
New_train_hash = categorial_feat_concat(X_train)
New_test_hash = categorial_feat_concat(X_test)


counts_hash_train = np.ndarray((New_train_hash.shape[0], 0), dtype=float)
counts_hash_test = np.ndarray((New_test_hash.shape[0], 0), dtype=float)
for i in xrange(X_train.shape[1]):
    #print New_Train.shape, categorial_feat_transorm(x, y_train, 3).shape
    t_train, t_test = categorial_feat_transorm(New_train_hash[:, i], y_train, New_test_hash[:, i], 3)
    counts_hash_train = np.hstack((counts_hash_train, t_train))
    counts_hash_test = np.hstack((counts_hash_test, t_test))
    
#counts_hash_test = normalize(counts_hash_test)
#counts_hash_train = normalize(counts_hash_train)

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
for k in range(1, 25):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(counts_hash_train, y_train)
    t_res = neigh.predict_proba(counts_hash_test)[:, 1]
    #print t_res.shape, y_test.shape
    print "k = " + str(k)
    print roc_auc_score(y_test, t_res)

k = 1
0.669245072859
k = 2
0.710560134596
k = 3
0.721695938024
k = 4
0.735321550031
k = 5
0.744991539198
k = 6
0.7565118347
k = 7
0.759017637574
k = 8
0.765371955423
k = 9
0.762708545842
k = 10
0.770566776722
k = 11
0.769948570254
k = 12
0.772768777842
k = 13
0.773322737562
k = 14
0.775602913046
k = 15
0.773233870382
k = 16
0.7736969861
k = 17
0.775098104335
k = 18
0.775729878997
k = 19
0.775654490156
k = 20
0.776281861894
k = 21
0.776492663111
k = 22
0.77568558019
k = 23
0.774773815506
k = 24
0.771463894945


В итоге получилось, что KNN оказался не очень чувствителен к переобучению по кликам

## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

Пробуем обучать одно дерево без фолдов со счетчиками

In [23]:
tree_X_train = categorial_feat_concat(X_train)
tree_X_test = categorial_feat_concat(X_test)
tree_counts_hash_train = np.ndarray((tree_X_train.shape[0], 0), dtype=float)
tree_counts_hash_test = np.ndarray((tree_X_test.shape[0], 0), dtype=float)
for i in xrange(X_train.shape[1]):
    #print New_Train.shape, categorial_feat_transorm(x, y_train, 3).shape
    t_train, t_test = categorial_feat_transorm(tree_X_train[:, i], y_train, tree_X_test[:, i], 1)
    tree_counts_hash_train = np.hstack((tree_counts_hash_train, t_train))
    tree_counts_hash_test = np.hstack((tree_counts_hash_test, t_test))


In [24]:
from sklearn.tree import DecisionTreeClassifier

depth = [1, 2, 3, 4, 5, 10, 50, 100]
ms_leaf = [1, 2, 5, 10]

for d in depth:
    for ms in ms_leaf:
        print "max_depth = " + str(d)
        print "min_samples_leaf = " + str(ms)
        clf = DecisionTreeClassifier(max_depth=d, min_samples_leaf=ms)
        clf.fit(tree_counts_hash_train, y_train)
        tree_res = clf.predict_proba(tree_counts_hash_test)[:, 1]
        print roc_auc_score(y_test, tree_res)
        


max_depth = 1
min_samples_leaf = 1
0.66002615876
max_depth = 1
min_samples_leaf = 2
0.66002615876
max_depth = 1
min_samples_leaf = 5
0.66002615876
max_depth = 1
min_samples_leaf = 10
0.66002615876
max_depth = 2
min_samples_leaf = 1
0.675540445439
max_depth = 2
min_samples_leaf = 2
0.675540445439
max_depth = 2
min_samples_leaf = 5
0.675540445439
max_depth = 2
min_samples_leaf = 10
0.675540445439
max_depth = 3
min_samples_leaf = 1
0.70126026061
max_depth = 3
min_samples_leaf = 2
0.701097891557
max_depth = 3
min_samples_leaf = 5
0.701092050943
max_depth = 3
min_samples_leaf = 10
0.701092050943
max_depth = 4
min_samples_leaf = 1
0.669022320848
max_depth = 4
min_samples_leaf = 2
0.668816192121
max_depth = 4
min_samples_leaf = 5
0.668676826099
max_depth = 4
min_samples_leaf = 10
0.668676826099
max_depth = 5
min_samples_leaf = 1
0.673939218803
max_depth = 5
min_samples_leaf = 2
0.673804075993
max_depth = 5
min_samples_leaf = 5
0.673652040333
max_depth = 5
min_samples_leaf = 10
0.684989210141


Вообще, интересно. При max_depth = 3 вобще не оказалось не важным кол-во объектов в листах. При этом там же и достигается лучший результат 0.701. Хотя при другой глубине результаты другие

#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [25]:
from sklearn.ensemble import RandomForestClassifier
n_est = [1, 2, 3, 5, 10, 15, 20, 35, 50, 100, 200, 500, 1000, 5000]
for n_estim in n_est:
    print "n_estimators = " + str(n_estim)
    clf = RandomForestClassifier(n_estimators = n_estim)
    clf.fit(tree_counts_hash_train, y_train)
    random_tree_res = clf.predict_proba(tree_counts_hash_test)[:, 1]
    print roc_auc_score(y_test, random_tree_res)
    
    

n_estimators = 1
0.637915483269
n_estimators = 2
0.682977702874
n_estimators = 3
0.719153024791
n_estimators = 5
0.709746761784
n_estimators = 10
0.750553780009
n_estimators = 15
0.745673093857
n_estimators = 20
0.745201711426
n_estimators = 35
0.759245331643
n_estimators = 50
0.760971906829
n_estimators = 100
0.765144980199
n_estimators = 200
0.769430283203
n_estimators = 500
0.768223612468
n_estimators = 1000
0.770077423172
n_estimators = 5000
0.769533976556


Получаем лучший результат на 1000 деревьев. Хотя он не сильно отличается от результата, например, на 200

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [39]:
tree_X_train = categorial_feat_concat(X_train)
tree_X_test = categorial_feat_concat(X_test)
tree_counts_hash_train_folding = np.ndarray((tree_X_train.shape[0], 0), dtype=float)
tree_counts_hash_test_folding = np.ndarray((tree_X_test.shape[0], 0), dtype=float)
for i in xrange(X_train.shape[1]):
    #print New_Train.shape, categorial_feat_transorm(x, y_train, 3).shape
    t_train, t_test = categorial_feat_transorm(tree_X_train[:, i], y_train, tree_X_test[:, i], 3)
    tree_counts_hash_train_folding = np.hstack((tree_counts_hash_train_folding, t_train))
    tree_counts_hash_test_folding = np.hstack((tree_counts_hash_test_folding, t_test))


In [40]:
n_est = [1, 2, 3, 5, 10, 15, 20, 35, 50, 100, 200, 500, 1000, 5000]


#tree_counts_hash_test_folding = normalize(tree_counts_hash_test_folding)
#tree_counts_hash_train_folding = normalize(tree_counts_hash_train_folding)
for n_estim in n_est:
    print "n_estimators = " + str(n_estim)
    clf = RandomForestClassifier(n_estimators = n_estim)
    clf.fit(tree_counts_hash_train_folding, y_train)
    random_tree_res = clf.predict_proba(tree_counts_hash_test_folding)[:, 1]
    print roc_auc_score(y_test, random_tree_res)

n_estimators = 1
0.531637075423
n_estimators = 2
0.577621033684
n_estimators = 3
0.627162554515
n_estimators = 5
0.645567765133
n_estimators = 10
0.692115746941
n_estimators = 15
0.715047253258
n_estimators = 20
0.722729996169
n_estimators = 35
0.766213453035
n_estimators = 50
0.771360740726
n_estimators = 100
0.791176863839
n_estimators = 200
0.787900549269
n_estimators = 500
0.792868125778
n_estimators = 1000
0.795329270423
n_estimators = 5000
0.793171118832


Вообще, результат заметно улучшился. Вообще, есть идея, что KNN считает расстояние, и вообще говоря, там выкидывай не выкидывай на фолдах, особо ничего не поменяется. А когда строятся деревья, они могут делать деления как раз по кликам. И вот тогда внесение этой лишний информации из теста может быть фатальным и будет происходить переобучение.